## Solution by: [Your Name] [No partners allowed]

## Demonstration and Homework: Generating Music
### Due by 10am Monday April 1

This notebook uses conditional probabilities to generate random sequences of music. This notebook is mostly for demonstration and the code has already been written, but there is a small part at the end that you need to complete yourself for a grade.

This notebook should be submitted as a file named `lastname.ipynb`, substituting `lastname` with your last name. 

**Collaboration:** No collaboration is allowed on this assignment.

**Late submissions:** Late assignments will receive 80% of credit up to one day late, and 50% of credit if two days late. Assignments will not be accepted after two days.

### Sampling from a distribution in Python

The function below is similar to the `sample` functions we've used in previous notebooks in class, but extended to more than two outcomes. It takes as input a *dictionary* where the keys are the outcomes and the value associated with each key is the probability of that outcome. A randomly chosen key will be returned as output.

In [ ]:
import random

def sample(distribution):
    assert round(sum(distribution.values()), 6) == 1.0, 'Not a valid distribution!'
    
    random_point = random.random()
    cumulative_probability = 0
    
    for outcome in distribution:
        cumulative_probability += distribution[outcome]
        if random_point < cumulative_probability:
            return outcome

You can try it in the cell below as a simple example. When you run the function, it will select one of the three colors in the dictionary according to their respective probabilities.

In [ ]:
colors = {}
colors['red'] = 0.5
colors['green'] = 0.3
colors['blue'] = 0.2

sample(colors)

### Generating MIDI music in Python

MIDI is a format for computer-generated music. You can control the pitch, octave, and duration (among other attributes) of notes. Various Python libraries exist for creating and editing MIDI files. 

We will use a library called [`pyknon`](https://github.com/kroger/pyknon/) which is a relatively small and simple library. You can install this library using `pip` or `conda`. If you are unable to install the library, you will not be penalized (you can still write code to generate notes), you just won't be able to listen to your results.

The code below will attempt to install it for you, though if it does not work, you will need to use a different method. 

In [ ]:
import sys
!{sys.executable} -m pip install pyknon

This library uses a string representation to define notes. The number after the pitch indicates the duration of the note (half note, quarter note, etc.), which can be additionally lengthened by adding dots after the number. To adjust the octave, add one or more ' or , to increase or decrease the octave from the default, respectively.

When you run the code below, it creates a short sequence of notes and writes it to a MIDI file named *output.mid*. The file will be created in whatever directory this notebook is saved in. When you open the file, you can hear the short song.

In [ ]:
from pyknon.genmidi import Midi
from pyknon.music import NoteSeq

notes1 = NoteSeq("E8 E4 E4 C8 E4 G2 G2, C4.' G4., E4., A4, B4, Bb8, A4,")
midi = Midi(1, tempo=200)
midi.seq_notes(notes1, track=0)
midi.write("output.mid")

You can directly open the MIDI file (with whatever your machine's default program is) by running the cell below.

In [ ]:
!open output.mid

### Building probabilities of note sequences

Different algorithms exist for automatically composing music, usually involving randomness and probabilities. We will use a relatively simple approach called a [Markov model](https://en.wikipedia.org/wiki/Markov_model). This is a probability model where the probability of a note depends on the previous note. In other words, for every pair of notes, we have a *conditional probability* of one note conditioned on the other.

Where do these probabilities come from? We can calculate them from an existing sequence of notes. For example, in the sequence in the `pyknon` cell above, E4 is followed by E4 once, followed by C8 once, and followed by G2 once. So the conditional probabilities would be estimated as: $P(E4|E4)=\frac{1}{3}, P(C8|E4)=\frac{1}{3}, P(G2|E4)=\frac{1}{3}$.

The function below takes a note sequence (written as a string) as input and calculates the conditional probabilities as they appear in the sequence. It returns a dictionary containing the conditional probabilities. You can then use this dictionary in combination with the `sample` function defined at the start of this notebook to randomly sample notes according to these probabilities.

You need to understand the structure of the dictionary used in this code. It is a nested dictionary (dictionary within a dictionary) that has two levels. The keys of the first level indicate the first note, and the keys of the second level indicate the next note. Thus, `cond['E4']['C8']` contains the value of $P(C8|E4)$.

In [ ]:
def calculate_conditional_probabilities(sequence):
    sample_space = set()      # set of possible outcomes (notes in the example sequence)
    notes = sequence.split()  # split the string sequence into a list of string tokens
    for note in notes:
        sample_space.add(note)
    
    # first initialize the dictionaries
    cond = {}
    denominator = {}
    for note1 in sample_space:
        cond[note1] = {}
        for note2 in sample_space:
            cond[note1][note2] = 0.0
        denominator[note1] = 0.0
    
    # count the number of times note1 is followed by note2;
    # also count the number of times note1 appears at all (for the denominator)
    for i in range(1, len(notes)):
        note1 = notes[i-1]   # previous note in list
        note2 = notes[i]     # current note in list
        
        cond[note1][note2] += 1.0
        denominator[note1] += 1.0
    
    # divide each value by the corresponding denominator;
    # also add a small value 0.1 to avoid probabilities of zero
    for note1 in sample_space:
        for note2 in sample_space:
            cond[note1][note2] = (cond[note1][note2] + 0.1) / (denominator[note1] + 0.1*len(sample_space))
        
    return cond

### Practice: generating random note sequences [6 points]

The code below produces a randomly generated sequence of 25 notes. It first randomly selects a note to begin the sequence using the `random.choice` function. The remaining 24 notes should be sampled from the conditional probabilities, where each note is sampled according to the probability conditioned on the previous note in the sequence.

The code below creates a variable called `new_note` which is currently simply set to the `previous_note`. You should change this line to instead use the `sample` function such that it randomly samples the value of `new_note` from the probability distribution conditioned on the value of `previous_note`.

You only need to change the one line, although it is allowable for your solution to add other lines.

In [ ]:
example_sequence = "E8 E4 E4 C8 E4 G2 G2, C4.' G4., E4., A4, B4, Bb8, A4,"
conditional_distribution = calculate_conditional_probabilities(example_sequence)
#print(conditional_distribution)

# begin by randomly selecting one of the notes from the sequence
starting_note = random.choice(list(conditional_distribution.keys())) 
output_sequence = [starting_note]   # list of notes that will be randomly selected

# populate the list with 25 notes
while len(output_sequence) < 25:    
    previous_note = output_sequence[-1]
    new_note = previous_note               # TODO: change this line!
    output_sequence.append(new_note)

# use the 'join' function to combine the list of 25 notes into a single string and print it out
random_sequence = ' '.join(output_sequence)
print(random_sequence)

Finally (and optionally), you can use the `pyknon` library again to create a MIDI file of your randomly generated sequence, which will allow you to hear your song!

In [ ]:
from pyknon.genmidi import Midi
from pyknon.music import NoteSeq

notes2 = NoteSeq(random_sequence)
midi = Midi(1, tempo=200)
midi.seq_notes(notes2, track=0)
midi.write("output2.mid")

In [ ]:
!open output2.mid